Setup

In [3]:
# Import Sentence Transformers from LlamaIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load the Embedding Model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

print("Sentence Transformer model loaded successfully!")

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sentence Transformer model loaded successfully!


In [4]:
from llama_index.llms.llama_cpp import LlamaCPP
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore
import psycopg2

In [2]:
from llama_index.llms.llama_cpp import LlamaCPP
llm = LlamaCPP(
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="/tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=2900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
    )

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /tmp/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

: 

In [1]:
# Step 2: Database Configuration
db_name = "vector_db"
host = "localhost"
user = "postgres"  
password = "postgres"  
port = "5432"

In [5]:
# Step 3: Initialize the Postgres Database
try:
    conn = psycopg2.connect(
        dbname="postgres",
        host=host,
        password=password,
        port=port,
        user=user,
    )
    conn.autocommit = True
    with conn.cursor() as c:
        c.execute(f"DROP DATABASE IF EXISTS {db_name}")
        c.execute(f"CREATE DATABASE {db_name}")
    print(f"Database '{db_name}' created successfully!")
except Exception as e:
    print(f"Error creating database: {e}")

Database 'vector_db' created successfully!


In [6]:
# Step 4: Set up Vector Store in Postgres
try:
    vector_store = PGVectorStore.from_params(
        database=db_name,
        host=host,
        password=password,
        port=port,
        user=user,
        table_name="llama2_paper",
        embed_dim=384,  # Adjust based on your embedding model
    )
    print("Vector store initialized successfully!")
except Exception as e:
    print(f"Error initializing vector store: {e}")

Vector store initialized successfully!


In [ ]:
Build an Ingestion Pipeline from Scratch

In [7]:
import wget

url = "https://arxiv.org/pdf/2307.09288.pdf"
output = "data/llama2.pdf"
wget.download(url, output)
print(f"Downloaded to {output}")


Downloaded to data/llama2.pdf


In [8]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

# Load PDF document
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

print(f"Loaded {len(documents)} document(s)!")

Loaded 77 document(s)!


In [9]:
from llama_index.core.node_parser import SentenceSplitter

# Text splitting
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",  # You can customize the separator if needed
)
text_chunks = []
doc_idxs = []

for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

print(f"Split document into {len(text_chunks)} text chunks!")

Split document into 107 text chunks!


In [10]:
#Construct Nodes from Text Chunks
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

print(f"Constructed {len(nodes)} nodes!")

Constructed 107 nodes!


In [12]:
# Generate Embeddings for Each Node
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

print("Embeddings generated for all nodes!")

Embeddings generated for all nodes!


In [14]:
#Store Nodes in the Vector Store
vector_store.add(nodes)
print("Nodes have been added to the vector store!")

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "public.data_llama2_paper" does not exist
LINE 1: INSERT INTO public.data_llama2_paper (text, metadata_, node_...
                    ^

[SQL: INSERT INTO public.data_llama2_paper (text, metadata_, node_id, embedding) SELECT p0::VARCHAR, p1::JSON, p2::VARCHAR, p3::VECTOR(384) FROM (VALUES (%(text__0)s, %(metadata___0)s::JSON, %(node_id__0)s, %(embedding__0)s, 0), (%(text__1)s, %(metadata___ ... 8585 characters truncated ... er) ORDER BY sen_counter RETURNING public.data_llama2_paper.id, public.data_llama2_paper.id AS id__1]
[parameters: {'node_id__0': 'cbfc2660-d930-4a25-967e-15cf5a4f17a1', 'metadata___0': '{"total_pages": 77, "file_path": "./data/llama2.pdf", "source": "1", "_node_content": "{\\"id_\\": \\"cbfc2660-d930-4a25-967e-15cf5a4f17a1\\", \\"emb ... (496 characters truncated) ... }\\\\n\\\\n{content}\\", \\"class_name\\": \\"TextNode\\"}", "_node_type": "TextNode", "document_id": "None", "doc_id": "None", "ref_doc_id"